In [ ]:
#Logistic
# import tqdm 
import argparse 
import traceback 
import datetime
import json
import os 
import sys
sys.path.append('.')
sys.path.append('./src')
import warnings
warnings.filterwarnings("ignore", message=r"Passing", category=FutureWarning)

# Main
import torch 
# import pytorch_lightning as pl 
from utils import data_util
from utils import util_torch
import numpy as np
from src.model_point_torch import GPointNet
import src.network_torch as network_torch
from metrics.evaluation_metrics import one_for_one_EMD_CD_
import multiprocessing as mp
import matplotlib.pyplot as plt
from itertools import product, combinations


In [ ]:
def parse_config():

    parser = argparse.ArgumentParser()

    # Point cloud related
    
    parser.add_argument('-rec_step_size', type=float, default=0.2, help='')
    parser.add_argument('-rec_step', type=int, default=64, help='')
    parser.add_argument('-batch_size', type=int, default=128, help='')
    parser.add_argument('-category', type=str, default="modelnet")
    parser.add_argument('-output_dir', type=str, default="default")
    parser.add_argument('-synthesis', action='store_true')
    parser.add_argument('-evaluation', action='store_true')
    parser.add_argument('-reconstruction', action='store_true')
    parser.add_argument('-intepolation', action='store_true')
    parser.add_argument('-visualize_layer', action='store_true')
    parser.add_argument('-cuda', type=str, default="0", help='')
    parser.add_argument('-max_num', type=int, default=4, help='')
    parser.add_argument('-visualize_mode', type=int, default=3)
    parser.add_argument('-checkpoint_path', type=str, default="output/pytorch/modelnet10_default_big_nlr5e4_epo_1200.ckpt")
    
    return parser.parse_args("")

In [ ]:
opt = parse_config()

In [ ]:
opt.category = "chair" 
opt.checkpoint_path = "checkpoint/syn_cvpr_chair.ckpt"
opt.synthesis = True 
opt.evaluation = True
#reconstruction 

In [ ]:
opt.max_num = 64

In [ ]:
opt.cuda = -2

In [ ]:
if opt.cuda == "-1":
    opt.cuda = 1
elif opt.cuda == "-2":
    opt.cuda = None
else: 
    os.environ["CUDA_VISIBLE_DEVICES"] = str(opt.cuda)
opt.data_path = "data/%s_train.npy" % opt.category
opt.output_dir = opt.checkpoint_path[:-5] if opt.output_dir=="default" else opt.output_dir
print(opt.output_dir)

In [ ]:
current_step = 0
if not os.path.exists(opt.output_dir):
    os.makedirs(opt.output_dir)
model = GPointNet.load_from_checkpoint(opt.checkpoint_path)#.cuda()

In [ ]:
syn_pcs = model(batch_size=min(model.C.batch_size, opt.max_num)).cpu().data.numpy()
#util_torch.visualize(syn_pcs[:20], num_rows=5, num_cols=4, output_filename="%s/syn_%d.png" % (opt.output_dir, current_step), mode=1)
#np.save("%s/syn_%d.npy" % (opt.output_dir, current_step), syn_pcs)

In [ ]:
syn_pcs.shape

In [ ]:
model.C.batch_size

In [ ]:
ref_pcs = np.load("data/%s_test.npy" % opt.category)

In [ ]:
ref_pcs.shape

In [ ]:
res = util_torch.quantitative_analysis(syn_pcs, ref_pcs, model.C.batch_size, full=False)

In [ ]:
res

In [ ]:
# res

In [ ]:
syn_pcs.device